In [3]:
!pip3 install playwright

In [4]:
import json
from bs4 import BeautifulSoup
from playwright.sync_api import sync_playwright
import threading

ModuleNotFoundError: No module named 'playwright.sync_api'; 'playwright' is not a package

In [42]:
import ollama

def talk(system, prompt):
    messages = [
        {
            'role': 'system',
            'content': system
        },
        {
            'role': 'user',
            'content': prompt
        }
    ]

    response = ollama.chat(model='llama3', messages=messages)

    return response['message']['content']



In [46]:
import os
os.environ["PWDEBUG"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [43]:
from sentence_transformers import SentenceTransformer
import torch
dev = "mps"

model = SentenceTransformer('all-MiniLM-L6-v2')
tens = torch.tensor(model.encode(["Hello world"])).to(dev)

/Users/leo/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
import re

stri = "thioasdfpaoidfgsaefgsef RES: hello world"

def findRES(stri):
    match = re.search("RES:(.*)", stri)
    payload = match.group(1)
    return payload

findRES(stri)

' hello world'

In [48]:
system = "You are an expert at extracting information from text data, what is given to you surrounded by ***. Always explain your reasoning, and then end your response with RES: and then your answer."
def find_person(college, person, putter):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False, slow_mo=50)
        page = browser.new_page()
        page.goto("https://www.google.com/")
        page.wait_for_load_state('load')
        page.fill('#APjFqb', college + " " + person)
        page.press('#APjFqb', 'Enter')

        page.wait_for_selector('h3')
        search_results = page.query_selector_all('h3')
        titles = [result.inner_text() for result in search_results]
        target = college + "Office of " + person
        tens = torch.tensor(model.encode([target] + titles)).to(dev)
        tens /= torch.norm(tens, dim=1, keepdim=True)
        sims = tens[1:] @ tens[0]
        best_pos = torch.argmax(sims).item()
        search_results[best_pos].click()

        page.wait_for_load_state('load')
        page_content = page.content()        
        browser.close()
        soup = BeautifulSoup(page_content, 'html.parser')
        text_data = soup.get_text()
        prompt = f"Use this data to find me the {person} of {college}: ***{text_data}***"
        raw_out = talk(system, prompt)
        payload = findRES(raw_out)
        print(payload)
        # putter.append({"College":college,"Position":person,"Name":payload})



In [1]:
with open("name_wiki_link.json", "r") as file:
    data = json.load(file)

colleges = [i["Name"] for i in data]
js = []
for college in [colleges[0]]:
    thread = threading.Thread(target=find_person, args=(college, "President", js))
    thread.start()
    thread.join()
    exit()
with open("presidents.json", "w") as file:
    json.dump(js, file, indent=4)

NameError: name 'json' is not defined